In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

**<font size="5" color="red">ch4. RNN(Recurrent Neural Network 순환신경망</font>**
# 1. 문맥을 이용하여 모델만들기

In [2]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [3]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]
print(encoded)
print(t.word_index)

[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [8]:
# 문자열 리스트을 인덱스 시퀀스로 반환
print(t.texts_to_sequences(['경마장에 말이 있다','말이 뛰다']))
print(t.texts_to_sequences(['경마장에 말이 있다'])[0])

[[2, 1, 5], [1]]
[2, 1, 5]


In [11]:
print(t.texts_to_matrix(['말이 뛰고']))
print(t.texts_to_sequences(['말이 뛰고']))

[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[1, 4]]


In [ ]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [23]:
print(t.word_index)
for word, value in t.word_index.items():
    if value==2:
        print("2:", word)
        break

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}
2: 경마장에


In [26]:
# text를 학습시키기 위해 ['경마장에 있는', '경마장에 있는 말이', '경마장에 있는 말이 뛰고',...]
sequences = []
for line in text.split('\n'):
    print('원래 문장 :', line)
    encoded = t.texts_to_sequences([line])[0]
    print('encoded된 문장 :', encoded)
    for i in range(0, len(encoded)-1): # 시작 index(빨간색 화살표)
        for j in range(i+2, len(encoded)+1): # 끝나는 index(파란색 화살표)
            sequences.append(encoded[i:j])
# sequences
print('sequences와 해석을 출력')
for sequence in sequences:
    print('[', end='')
    for word_seq in sequence:
        for word, value in t.word_index.items():
            if word_seq==value:
                print("{}:{}".format(word_seq, word), end=' ')
                break
    print(']')

원래 문장 : 경마장에 있는 말이 뛰고 있다
encoded된 문장 : [2, 3, 1, 4, 5]
원래 문장 : 그의 말이 법이다
encoded된 문장 : [6, 1, 7]
원래 문장 : 가는 말이 고와야 오는 말이 곱다
encoded된 문장 : [8, 1, 9, 10, 1, 11]
sequences와 해석을 출력
[2:경마장에 3:있는 ]
[2:경마장에 3:있는 1:말이 ]
[2:경마장에 3:있는 1:말이 4:뛰고 ]
[2:경마장에 3:있는 1:말이 4:뛰고 5:있다 ]
[3:있는 1:말이 ]
[3:있는 1:말이 4:뛰고 ]
[3:있는 1:말이 4:뛰고 5:있다 ]
[1:말이 4:뛰고 ]
[1:말이 4:뛰고 5:있다 ]
[4:뛰고 5:있다 ]
[6:그의 1:말이 ]
[6:그의 1:말이 7:법이다 ]
[1:말이 7:법이다 ]
[8:가는 1:말이 ]
[8:가는 1:말이 9:고와야 ]
[8:가는 1:말이 9:고와야 10:오는 ]
[8:가는 1:말이 9:고와야 10:오는 1:말이 ]
[8:가는 1:말이 9:고와야 10:오는 1:말이 11:곱다 ]
[1:말이 9:고와야 ]
[1:말이 9:고와야 10:오는 ]
[1:말이 9:고와야 10:오는 1:말이 ]
[1:말이 9:고와야 10:오는 1:말이 11:곱다 ]
[9:고와야 10:오는 ]
[9:고와야 10:오는 1:말이 ]
[9:고와야 10:오는 1:말이 11:곱다 ]
[10:오는 1:말이 ]
[10:오는 1:말이 11:곱다 ]
[1:말이 11:곱다 ]


In [32]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [3, 1],
 [3, 1, 4],
 [3, 1, 4, 5],
 [1, 4],
 [1, 4, 5],
 [4, 5],
 [6, 1],
 [6, 1, 7],
 [1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11],
 [1, 9],
 [1, 9, 10],
 [1, 9, 10, 1],
 [1, 9, 10, 1, 11],
 [9, 10],
 [9, 10, 1],
 [9, 10, 1, 11],
 [10, 1],
 [10, 1, 11],
 [1, 11]]

In [31]:
# sequence별 len
print([len(sequence) for sequence in sequences])

[2, 3, 4, 5, 2, 3, 4, 2, 3, 2, 2, 3, 2, 2, 3, 4, 5, 6, 2, 3, 4, 5, 2, 3, 4, 2, 3, 2]


In [33]:
my_len = max([len(sequence) for sequence in sequences])
my_len

6

In [41]:
# sequences를 훈련 가능하도록 6개열로 조정
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(sequences=sequences, 
                                maxlen=my_len, # 최대 열수
                                # padding='post',
                                padding='pre', # 최대 열수보다 작은 경우 0을 채울 위치
                                # truncating='post' # 최대 열수보다 많은 경우 데이터를 짜를 위치
                                )
type(padded_sequences), padded_sequences.shape, padded_sequences

(numpy.ndarray,
 (28, 6),
 array([[ 0,  0,  0,  0,  2,  3],
        [ 0,  0,  0,  2,  3,  1],
        [ 0,  0,  2,  3,  1,  4],
        [ 0,  2,  3,  1,  4,  5],
        [ 0,  0,  0,  0,  3,  1],
        [ 0,  0,  0,  3,  1,  4],
        [ 0,  0,  3,  1,  4,  5],
        [ 0,  0,  0,  0,  1,  4],
        [ 0,  0,  0,  1,  4,  5],
        [ 0,  0,  0,  0,  4,  5],
        [ 0,  0,  0,  0,  6,  1],
        [ 0,  0,  0,  6,  1,  7],
        [ 0,  0,  0,  0,  1,  7],
        [ 0,  0,  0,  0,  8,  1],
        [ 0,  0,  0,  8,  1,  9],
        [ 0,  0,  8,  1,  9, 10],
        [ 0,  8,  1,  9, 10,  1],
        [ 8,  1,  9, 10,  1, 11],
        [ 0,  0,  0,  0,  1,  9],
        [ 0,  0,  0,  1,  9, 10],
        [ 0,  0,  1,  9, 10,  1],
        [ 0,  1,  9, 10,  1, 11],
        [ 0,  0,  0,  0,  9, 10],
        [ 0,  0,  0,  9, 10,  1],
        [ 0,  0,  9, 10,  1, 11],
        [ 0,  0,  0,  0, 10,  1],
        [ 0,  0,  0, 10,  1, 11],
        [ 0,  0,  0,  0,  1, 11]]))

In [43]:
# 독립변수(X)와 타겟변수(종속변수 y)로 분리
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]
y

array([ 3,  1,  4,  5,  1,  4,  5,  4,  5,  5,  1,  7,  7,  1,  9, 10,  1,
       11,  9, 10,  1, 11, 10,  1, 11,  1, 11, 11])

In [48]:
# 단어 갯수
vocab_size = len(t.word_index)

In [49]:
# 원핫인코딩
from tensorflow.keras.utils import to_categorical
Y = to_categorical(y, 
                   vocab_size+1) # 원핫인코딩 카테고리 수
Y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.,

In [50]:
X.shape, Y.shape

((28, 5), (28, 12))